# An exploration on fitbit data -mood and stress parameters?

This EDA has been prepared by Esther Guiu Hernandez on March 2024

This is Lifesnaps Fitbit dataset 

## About the data files: 
- <b>  Scored Surveys </b> : CSV files containing scored version of PANAS, STAI surveys
- <b> Personality trait  </b> : CSV files containing personality trait data
- <b>  Fitbit & EMA Data (daily granularity) </b>: csv_rais_anonymized/daily_fitbit_sema_df_unprocessed.csv
- <b>  Fitbit & EMA Data (hourly granularity) </b>: csv_rais_anonymized/hourly_fitbit_sema_df_unprocessed.csv

 

In [169]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import missingno as msno
from tabulate import tabulate

In [170]:
# Load in the dataframe from fitbit data

#fitbit and sema
daily_fitbit_sema_df = pd.read_csv('data/daily_fitbit_sema_df_unprocessed.csv', delimiter=',')
hourly_fitbit_sema_df = pd.read_csv('data/hourly_fitbit_sema_df_unprocessed.csv', delimiter=',')

#survey data
stai_survey_df = pd.read_csv('data/stai.csv', delimiter=',')
personality_survey_df = pd.read_csv('data/personality.csv', delimiter=',')
panas_survey_df = pd.read_csv('data/panas.csv', delimiter=',')

C:\Users\egh22\AppData\Local\Temp\ipykernel_32804\1705282665.py:5: DtypeWarning: Columns (5,10,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  hourly_fitbit_sema_df = pd.read_csv('data/hourly_fitbit_sema_df_unprocessed.csv', delimiter=',')


In [171]:
hourly_fitbit_sema_df.columns

Index(['Unnamed: 0', 'id', 'date', 'hour', 'temperature', 'badgeType',
       'calories', 'distance', 'activityType', 'bpm', 'mindfulness_session',
       'scl_avg', 'steps', 'minutes_in_default_zone_1',
       'minutes_below_default_zone_1', 'minutes_in_default_zone_2',
       'minutes_in_default_zone_3', 'age', 'gender', 'bmi', 'step_goal',
       'min_goal', 'max_goal', 'step_goal_label', 'ALERT', 'HAPPY', 'NEUTRAL',
       'RESTED/RELAXED', 'SAD', 'TENSE/ANXIOUS', 'TIRED', 'ENTERTAINMENT',
       'GYM', 'HOME', 'HOME_OFFICE', 'OTHER', 'OUTDOORS', 'TRANSIT',
       'WORK/SCHOOL'],
      dtype='object')

In [172]:
daily_fitbit_sema_df.columns

Index(['Unnamed: 0', 'id', 'date', 'nightly_temperature', 'nremhr', 'rmssd',
       'spo2', 'full_sleep_breathing_rate', 'stress_score',
       'sleep_points_percentage', 'exertion_points_percentage',
       'responsiveness_points_percentage', 'daily_temperature_variation',
       'badgeType', 'calories', 'filteredDemographicVO2Max', 'distance',
       'activityType', 'bpm', 'lightly_active_minutes',
       'moderately_active_minutes', 'very_active_minutes', 'sedentary_minutes',
       'mindfulness_session', 'scl_avg', 'resting_hr', 'sleep_duration',
       'minutesToFallAsleep', 'minutesAsleep', 'minutesAwake',
       'minutesAfterWakeup', 'sleep_efficiency', 'sleep_deep_ratio',
       'sleep_wake_ratio', 'sleep_light_ratio', 'sleep_rem_ratio', 'steps',
       'minutes_in_default_zone_1', 'minutes_below_default_zone_1',
       'minutes_in_default_zone_2', 'minutes_in_default_zone_3', 'age',
       'gender', 'bmi', 'step_goal', 'min_goal', 'max_goal', 'step_goal_label',
       'ALERT',

# Analysing the fitbit data

What is the rleationship between the different stress associacted physiological data?  We are going to select the following variables

- <b>Age</b>
- <b>Gender</b>
- <b>BMI</b>
- <b>nightly_temperature </b>: measures the temperature at night during sleep
- <b>rmssd </b>: The Root Mean Square of Successive Differences (RMSSD) between heart beats. It measures short-term variability in the user’s heart rate while in deep sleep, in milliseconds (ms)
    -- Heart Rate Variability (HRV): If your nervous system is balanced, your heart is constantly being told to beat slower by your parasympathetic system, and beat faster by your sympathetic system. This causes a fluctuation in your heart rate: HRV.  higher HRV is seen as healthier, while a drop in HRV could indicate that you are experiencing stress or showing potential signs of illness. 
- <b> spo2 </b>: termine the percentage of oxygen saturation in the blood. A low score may be an indication of important changes in your fitness and wellness. Fitbit measures SpO2 above 80%, and a healthy score would be above 95%.
- <b> Full_sleep_breathing_rate </b> :  RR of healthy adults in a relax state is about 12–20 times per minute.
- <b> Strees score </b>:  Calculated by adding:
    - <b> responsiveness_points_percentage: </b>responsiveness out of a possible 30
    Responsiveness monitors your sympathetic nervous system, aka your fight or flight response, by monitoring your heart rate and heart rate variability. 
    - <b> exertion_points_percentage: </b> for exertion balance is out of 40 
     Exertion akes into account your recent physical activity like steps and accounts for both overexertion or lack of exercise. 
    - <b> sleep_points_percentage:</b> sleep patterns is out of 30 
    include measurements of deep sleep from the previous night and whether your sleep was fitful or fragmented. It also tracks your “sleep reservoir” based on the amount and quality of sleep you’ve managed over the previous week
    
-  <b> Sleep score </b>: Sleep Score that's made up of time asleep (50 percent of the score) the amount of time you spent in deep and REM sleep (25 percent of the score) and restoration (which shows how much of your sleep time is below your resting heart rate (also 25 percent).
- <b> daily_tempearature_variation: </b> The top-end Fitbit smartwatches feature a sensor that tracks your skin temperature each night to show how it varies from your personal baseline (set over three nights when you first setup the watch), so you can be aware of your trends over time. 

- <b> daily_temperature_variation:</b> variations in temperature can be caused by menstrual cycle in women, by ilness and also by stress periods. 

- <b>FilteredVO2 Max (Cardio Fitness Score):</b>  It’s a measurement of your cardiovascular fitness, or how well your body uses oxygen when you are working out at your hardest. The higher your score, the more fit you are. The more stationary your lifestyle, the lower your score will be, and the higher your risk of developing high blood pressure and coronary heart disease

- <b> lightly_active_minutes</b>
- <b> moderately_active_minutes</b>
- <b> very_active_minutes</b>
- <b> sedentary_minutes</b>
- <b> mindfulness_session</b>
- <b> scl_avg (skin conductance level): </b>
- <b>resting_hr ( Resting Heart Rate): </b> number of times your heart beats per minute (bpm) when you are still and rested .
RHR usually ranges from 60-100bpm, but varies according to your age and fitness level. Generally, the lower the better –
- <b> sleep_duration</b>
- <b> sleep_efficiency</b>
- <b> SEMA Values</b>: ALTERT, HAPPY, NEUTRAL, RESTED/RELAXED, TENSE/ANXIOUS, TIRED



# Data Pre-processing
<b> Daily_fitbit_sema_df </b>
1. Calculate which day of the week each day corresponds
2. Order by day accross participants and asign each day whether is day 1 in the study, so create new variable called Days_in_study
3. Delete all the features that we don't need
4. Save data in a new csv file

<b> Hourly_fitbit_sema_df </b>
1. Calculate which day of the week each day corresponds
2. Calculate whether the hour corresponds to the night (0-5) morning (6 -12), afternoon (13 - 18) evening (19 - 23)
3. Calculate a variable that reflects the number of days in the study in order to align participants accross this time
4. Delete all the features that we don't need
5. Save data in a new csv file

<b>  Scored Surveys </b>
- PANAS
1. Add a column that represents the first day in the study as the minimum date in the hourly or daily fitbit dataset
2. Calculate number of days in the study and create a new column that reflects it, this way we can compare participants in a way

- STAI
1. Add a column that represents the first day in the study as the minimum date in the hourly or daily fitbit dataset
2. Calculate number of days in the study and create a new column that reflects it, this way we can compare participants in a way


<b> Personality trait  </b>
- No pre-processing needed

# Different types of analysis
<b> Hourly_fitbit_sema_df </b>
Create new data frame averaged accross this times.
    - For distance and steps we will calculate the sum, as we want to see the total they have done
    - For calories we will calculate the maximum    
- Look at whether SEMA values changes depending on time of the day and day of the week
- Look at whether the amount that someone has moved (as distance) has any correlation with the way the feel
- Look at whether the maximum number of calories they have burnt has any relationship with the way the feel
- Separate analysis accross age groups 
- Separate analysis accross males/ females 

<b> Daily_fitbit_sema_df </b>
- Look at whether day of the week is correlated with measures of stress score
- Look whether day of the week is correlated with sleep efficiency
- Look at whether sleep efficiency is correlated with stress score 
- Look at whether sedentary minutes are correlated with stress score 
- Look at whether stress score is correlated at all with SEMA mood

<b>  Scored Surveys </b>

<b> Personality trait  </b>


For females:
- An idea could be to look at daily temperature variation accross females and see how this relates to their stress score and to their sema values,
- From the graph we could look at the peaks and see whther the change  makes sense with something in their menstrual cycle, when temparature goes from high to low and has the higuest change this could mean menstruation





# 1. Data cleaning

## 1.1 Hourly fitbit sema df

In [173]:
columns_to_drop = ['minutes_below_default_zone_1', 'minutes_in_default_zone_2',
       'minutes_in_default_zone_3', 'step_goal', 'min_goal', 'max_goal', 'bpm', 'temperature', 'badgeType', 'scl_avg']

hourly_fitbit_sema_df.drop(columns=columns_to_drop, inplace = True)


# Create a mapping dictionary for Participant_ID to new integer numbers in order, so that is easier to plot later
unique_participants = hourly_fitbit_sema_df['id'].unique()
participant_mapping = {participant_id: i for i, participant_id in enumerate(sorted(unique_participants), 1)}

# Map Participant_ID to new integer numbers
hourly_fitbit_sema_df['Mapped_ID'] = hourly_fitbit_sema_df['id'].map(participant_mapping)

# Create a column that represents the number of day since the start of the study
# Calculate the minimum date for each participant
hourly_fitbit_sema_df['date'] = pd.to_datetime(hourly_fitbit_sema_df['date'])
hourly_fitbit_sema_df['Min_date'] = hourly_fitbit_sema_df.groupby('id')['date'].transform('min')
# substract the start date in the study from the current day to calculate which day from the study each entrry represents
hourly_fitbit_sema_df['Days_from_beggining'] = (hourly_fitbit_sema_df['date'] - hourly_fitbit_sema_df['Min_date']).dt.days + 1  # Adding 1 to start counting from day 1


# Save DataFrame to CSV file
hourly_fitbit_sema_df.to_csv('data/cleaned/hourly_fitbit_sema_cleaned.csv', index=False)  # Set index=False to exclude row numbers in the output CSV file



## 1.2 PANAS

In [174]:
## 1.2 PANAS

# create a dataframe that is only users and their start date, just once
users_and_start_date = hourly_fitbit_sema_df.drop_duplicates(subset=['id'], keep='first')[["id", "Min_date", "Mapped_ID"]]
#users_and_start_date.head(40)

# rename id column so that both dataframes can be merged
panas_survey_df.rename(columns={'user_id': 'id'}, inplace=True)

# Merge the two dataframes on Participant_ID
panas_survey_df = pd.merge(panas_survey_df, users_and_start_date, on='id', how='left')

# Convert 'Date' column to datetime
panas_survey_df['submitdate'] = pd.to_datetime(panas_survey_df['submitdate'])

# substract the start date in the study from the current day to calculate which day from the study each entrry represents
panas_survey_df['Days_from_beggining'] = (panas_survey_df['submitdate'] - panas_survey_df['Min_date']).dt.days + 1  # Adding 1 to start counting from day 1

# Calculate the number of weeks from the beginning of the study
panas_survey_df['Weeks_from_beggining'] = panas_survey_df['Days_from_beggining'] // 7  

# create a score that represents positive over negative
panas_survey_df['Positive_over_negative'] = (panas_survey_df['positive_affect_score'] / panas_survey_df['negative_affect_score']).round(2)

# Drop the 'Min_Date' column if not needed
panas_survey_df.drop(columns=['Min_date'], inplace=True)

# Map Participant_ID to new integer numbers
#panas_survey_df.head(10)

# Save DataFrame to CSV file
panas_survey_df.to_csv('data/cleaned/panas_survey_cleaned.csv', index=False)  # Set index=False to exclude row numbers in the output CSV file


## 1.3 STAI


In [175]:
# rename id column so that both dataframes can be merged
stai_survey_df.rename(columns={'user_id': 'id'}, inplace=True)

# Merge the two dataframes on Participant_ID
stai_survey_df = pd.merge(stai_survey_df, users_and_start_date, on='id', how='left')

# Convert 'Date' column to datetime
stai_survey_df['submitdate'] = pd.to_datetime(stai_survey_df['submitdate'])

# substract the start date in the study from the current day to calculate which day from the study each entrry represents
stai_survey_df['Days_from_beggining'] = (stai_survey_df['submitdate'] - stai_survey_df['Min_date']).dt.days + 1  # Adding 1 to start counting from day 1

# Calculate the number of weeks from the beginning of the study
stai_survey_df['Weeks_from_beggining'] = stai_survey_df['Days_from_beggining'] // 7  

# Drop the 'Min_Date' column if not needed
stai_survey_df.drop(columns=['Min_date'], inplace=True)

# Map Participant_ID to new integer numbers
#panas_survey_df.head(10)

# Save DataFrame to CSV file
stai_survey_df.to_csv('data/cleaned/stai_survey_df_cleaned.csv', index=False)  # Set index=False to exclude row numbers in the output CSV file


## 1.4 Daily fitbit sema

In [176]:
columns_to_drop = ['nremhr', 'badgeType',
       'bpm', 'step_goal', 'min_goal', 'max_goal', 'minutesAwake', 'minutesAfterWakeup', 'badgeType', 'sleep_deep_ratio' ,'sleep_wake_ratio',
                  'sleep_light_ratio', 'sleep_rem_ratio']

daily_fitbit_sema_df.drop(columns=columns_to_drop, inplace = True)


# Map Participant_ID to new integer numbers
daily_fitbit_sema_df['Mapped_ID'] = daily_fitbit_sema_df['id'].map(participant_mapping)

# Create a column that represents the number of day since the start of the study
# Calculate the minimum date for each participant
daily_fitbit_sema_df['date'] = pd.to_datetime(daily_fitbit_sema_df['date'])
daily_fitbit_sema_df['Min_date'] = daily_fitbit_sema_df.groupby('id')['date'].transform('min')
# substract the start date in the study from the current day to calculate which day from the study each entrry represents
daily_fitbit_sema_df['Days_from_beggining'] = (daily_fitbit_sema_df['date'] - daily_fitbit_sema_df['Min_date']).dt.days + 1  # Adding 1 to start counting from day 1


# Save DataFrame to CSV file
daily_fitbit_sema_df.to_csv('data/cleaned/daily_fitbit_sema_cleaned.csv', index=False)  # Set index=False to exclude row numbers in the output CSV file



## Data pre-processing

In [ ]:
###  Hourly_fitbit_sema_df ###

# Calculate which day of the week each day corresponds, and create a new variable called Day_of_the_weeek that stores this value
hourly_fitbit_sema_df.dtypes

# Calculate whether the hour corresponds to the night (0-5) morning (6 -12), afternoon (13 - 18) evening (19 - 23)
hourly_fitbit_sema_df.loc[(hourly_fitbit_sema_df['hour'] >= 0) & (hourly_fitbit_sema_df['hour'] <= 5), 'time_of_day'] = 'Night'
hourly_fitbit_sema_df.loc[(hourly_fitbit_sema_df['hour'] >= 6) & (hourly_fitbit_sema_df['hour'] <= 12), 'time_of_day'] = 'Morning'
hourly_fitbit_sema_df.loc[(hourly_fitbit_sema_df['hour'] >= 13) & (hourly_fitbit_sema_df['hour'] <=18), 'time_of_day'] = 'Afternoon'
hourly_fitbit_sema_df.loc[(hourly_fitbit_sema_df['hour'] >= 19) & (hourly_fitbit_sema_df['hour'] <= 23), 'time_of_day'] = 'Evening'


# Agggregation to look at time of day
#. Create new data frame averaged accross this times.
   # - For distance and steps we will calculate the sum, as we want to see the total they have done
   # - For calories we will calculate the maximum
hourly_fitbit_sema_by_time_of_day = hourly_fitbit_sema_df[['id','date','age', 'gender', 'bmi', 'calories', 'distance', 'ALERT', 'HAPPY', 'NEUTRAL',
                                              'RESTED/RELAXED','SAD', 'TENSE/ANXIOUS', 'TIRED',  'time_of_day' ]]

# define how do we want to aggregate each column
aggregations = {
    'calories': 'mean',  
    'distance': 'sum',
    'ALERT': 'max',
    'HAPPY': 'max',
    'NEUTRAL': 'max',
    'RESTED/RELAXED': 'max',
    'TENSE/ANXIOUS': 'max',
    'TIRED': 'max'}
    
hourly_fitbit_sema_df_summarized =  hourly_fitbit_sema_df2b.groupby(['id', 'date', "time_of_day", "age","gender","bmi"]).max().reset_index()

hourly_fitbit_sema_df_summarized['median_calories']=  hourly_fitbit_sema_df2b.groupby(['id', 'date', "time_of_day"]).median().reset_index()['calories']
hourly_fitbit_sema_df_summarized['total_sum_distance'] =  hourly_fitbit_sema_df2b.groupby(['id', 'date', "time_of_day"]).sum().reset_index()['distance']
hourly_fitbit_sema_df_summarized.drop(['calories', 'distance'], axis=1)
 

# Calculate day of the week and month
hourly_fitbit_sema_df_summarized['date'] = pd.to_datetime(hourly_fitbit_sema_df_summarized['date'])
hourly_fitbit_sema_df_summarized['day_of_week'] = hourly_fitbit_sema_df_summarized['date'].dt.weekday.map({0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'})
hourly_fitbit_sema_df_summarized['month'] = hourly_fitbit_sema_df_summarized['date'].dt.month_name()

# Calculate whether the day is a weekend or a weekday
hourly_fitbit_sema_df_summarized['day_type'] = hourly_fitbit_sema_df_summarized['date'].dt.weekday.map({0: 'WeekDay', 1: 'WeekDay', 2: 'WeekDay', 3: 'WeekDay', 4: 'WeekDay', 5: 'Weekend', 6: 'Weekend'})


#hourly_fitbit_sema_df_summarized.head(5)

# create a dataframe that is only users and their start date, just once
users_and_start_date = hourly_fitbit_sema_df_summarized.drop_duplicates(subset=['id'], keep='first')[["id", "Min_date", "Mapped_ID"]]
#users_and_start_date.head(40)

# Drop the 'Min_Date' column if not needed
hourly_fitbit_sema_df_summarized.drop(columns=['Min_date'], inplace=True)


In [ ]:
# Summaryse the amount of time users were enrolled in the study
enrollment_duration = hourly_fitbit_sema_df_summarized.groupby('id')['date'].apply(lambda x: x.max() - x.min())

# Calculate duration in months and days
enrollment_duration_months = enrollment_duration.dt.days // 30
enrollment_duration_days = enrollment_duration.dt.days 

description_months = enrollment_duration_months.describe().round(2)
description_days = enrollment_duration_days.describe().round(2)

print(description_months)
print(description_days)

## 1.2 PLotting the data

In [ ]:
#look at the missing values by time of day and overall

#missing SEMA, calories and distance values overall
msno.bar(hourly_fitbit_sema_df_summarized)

#missing SEMA, calories and distance by time of day, we can see there is quite a lot of misisng values compared to the amount of values we should have, but it is comparable accross time of day
# we should not have SEMA values at night , but we can see we can 25 values 
hourly_fitbit_sema_df_summarized.groupby("time_of_day").count()

In [ ]:
# drop na values from the mood columns as we are just interest on those who had values
hourly_fitbit_sema_df_summarized_no_na = hourly_fitbit_sema_df_summarized.dropna(subset=['ALERT'])[['id','date','calories', 'distance', 'ALERT', 'HAPPY', 'NEUTRAL',
                                              'RESTED/RELAXED','SAD', 'TENSE/ANXIOUS', 'TIRED',  'time_of_day', 'day_of_week' ]]

# Check which mood rows have been added incorrectly as user replied twice withn the same time frame
# We are adding all mood columns and they always would need to add one as they replied to one mood per timeframe, so if it is more than one it means than when collapsing 
# accross timeframes we did not do it right
column_names = [ 'ALERT', 'HAPPY', 'NEUTRAL','RESTED/RELAXED','SAD', 'TENSE/ANXIOUS', 'TIRED']
hourly_fitbit_sema_df_summarized_no_na['MOOD ADDED'] = hourly_fitbit_sema_df_summarized_no_na[column_names].sum(axis=1)

print('Number of rows removing na and before removing two mood instances in a row is: ' + str(len(hourly_fitbit_sema_df_summarized_no_na)))

# delete rows where MOOD ADDED is bigger than 1
hourly_fitbit_sema_df_summarized_no_na = hourly_fitbit_sema_df_summarized_no_na.drop(hourly_fitbit_sema_df_summarized_no_na[hourly_fitbit_sema_df_summarized_no_na['MOOD ADDED'] > 1].index)
print('Number of rows removing na and removing two mood instances in a row is: ' + str(len(hourly_fitbit_sema_df_summarized_no_na)))

#create a single variable that represents the mood that is the combination of the dummy variables
hourly_fitbit_sema_df_summarized_no_na['MOOD'] = pd.from_dummies(hourly_fitbit_sema_df_summarized_no_na[column_names])

msno.bar(hourly_fitbit_sema_df_summarized_no_na)

hourly_fitbit_sema_df_summarized_no_na.head(5)




In [ ]:
hourly_fitbit_sema_df_to_plot = hourly_fitbit_sema_df_summarized_no_na[["day_of_week", "time_of_day", "MOOD", "RESTED/RELAXED", "TIRED", "TENSE/ANXIOUS","HAPPY",  "NEUTRAL","ALERT","SAD"]]

hourly_fitbit_sema_df_to_plot = hourly_fitbit_sema_df_to_plot.groupby("time_of_day").sum().reset_index()

#order labels of day_of_week
weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
hourly_fitbit_sema_df_summarized_no_na.day_of_week = pd.Categorical(hourly_fitbit_sema_df_summarized_no_na.day_of_week,categories=weekdays)
hourly_fitbit_sema_df_summarized_no_na = hourly_fitbit_sema_df_summarized_no_na.sort_values('day_of_week')


#remove night as we were interested during the timeframes morning, afternon and evening
hourly_fitbit_sema_df_to_plot =  hourly_fitbit_sema_df_to_plot[hourly_fitbit_sema_df_to_plot.time_of_day != "Night"]

#order labels of time_of_day
timeday = ["Morning", "Afternoon", "Evening"]
hourly_fitbit_sema_df_to_plot.time_of_day = pd.Categorical(hourly_fitbit_sema_df_to_plot.time_of_day,categories=timeday)
hourly_fitbit_sema_df_to_plot = hourly_fitbit_sema_df_to_plot.sort_values('time_of_day')



#plot mood based on time of the day and day of the week
# STILL TODO as nor working properly
fig, ax = plt.subplots(figsize=(20, 10))
hourly_fitbit_sema_df_to_plot.plot(x="time_of_day", y=['RESTED/RELAXED','TIRED', 'TENSE/ANXIOUS','NEUTRAL', 'ALERT', 'HAPPY', 'SAD' ], kind="bar",ax=ax, stacked=True)
plt.xlabel('Time of day',fontsize=20)
plt.ylabel('Number of answers',fontsize=20)
plt.title('Mood answers by time of day',fontsize=20)
plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.grid()
plt.xticks(fontsize=18, rotation=0)
plt.yticks(fontsize=18, rotation=0)
plt.show()

In [ ]:
hourly_fitbit_sema_df_to_plot = hourly_fitbit_sema_df_summarized_no_na[["day_of_week", "time_of_day", "MOOD", "RESTED/RELAXED", "TIRED", "TENSE/ANXIOUS","HAPPY",  "NEUTRAL","ALERT","SAD"]]
hourly_fitbit_sema_df_to_plot = hourly_fitbit_sema_df_to_plot.groupby(["day_of_week"]).sum().reset_index()
#remove night as we were interested during the timeframes morning, afternon and evening
hourly_fitbit_sema_df_to_plot =  hourly_fitbit_sema_df_to_plot[hourly_fitbit_sema_df_to_plot.time_of_day != "Night"]


fig, ax = plt.subplots(figsize=(20, 10))

hourly_fitbit_sema_df_to_plot.plot(x="day_of_week", y=['RESTED/RELAXED','TIRED', 'TENSE/ANXIOUS','NEUTRAL', 'ALERT', 'HAPPY', 'SAD' ], kind="bar",ax=ax, stacked=True)
plt.xlabel('Week day',fontsize=20)
plt.ylabel('Number of answers',fontsize=20)
plt.title('Mood by day',fontsize=20)
plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.grid()
plt.xticks(fontsize=18, rotation=0)
plt.yticks(fontsize=18, rotation=0)
plt.show()



In [ ]:
#TODO: I wanted to stack both plots together but it seems it is not possible
#plot mood based on time of the day and day of the week
# STILL TODO as nor working properly
# TODO: how to normalize?
hourly_fitbit_sema_df_normalized = hourly_fitbit_sema_df_summarized_no_na.groupby(['day_of_week', 'MOOD']).size().groupby(level=0).apply(
    lambda x: 100 * x / x.sum()
).unstack()

# TODO: how to obtain proper x axis on this graph
hourly_fitbit_sema_df_normalized.plot(kind='bar',stacked=True)
plt.xlabel('Week day',fontsize=20)
plt.ylabel('Percentage of answers',fontsize=20)
plt.title('Normalized mood by day',fontsize=20)
plt.grid()
plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.xticks(fontsize=18, rotation=0)
plt.yticks(fontsize=18, rotation=0)
plt.show()